# After SOLO doublet removal


In [ ]:
# Python packages
import scanpy as sc
import scvi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [ ]:
path = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/solo_2019"

In [ ]:
adata_solo = sc.read_h5ad(f"{path}/adata_nodoublet.h5ad")

In [ ]:
adata_solo

In [ ]:
adata_solo.var.sort_values("n_cells_by_counts")

In [ ]:
adata_solo.obs

In [ ]:
adata_solo.obs

In [ ]:
def update_columns(row):
    if "10mix1" in row['sample_id']:
        row['condition'] = "10mix"
        row['batch_id'] = "1"
    elif "10mix2" in row['sample_id']:
        row['condition'] = "10mix"
        row['batch_id'] = "2"
    elif "11mix1" in row['sample_id']:
        row['condition'] = "11mix"
        row['batch_id'] = "1"
    elif "11mix2" in row['sample_id']:
        row['condition'] = "11mix"
        row['batch_id'] = "2"
    elif "GF1" in row['sample_id']:
        row['condition'] = "GF"
        row['batch_id'] = "1"
    elif "GF2" in row['sample_id']:
        row['condition'] = "GF"
        row['batch_id'] = "2"
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
adata_solo.var.mt.value_counts()

In [ ]:
adata_solo.obs.is_doublet.value_counts()

In [ ]:
sc.pl.umap(adata_solo, color=["leiden"], wspace=1)

In [ ]:
sc.pl.umap(adata_solo, color=["batch_id"], wspace=1)

In [ ]:
sc.pl.umap(adata_solo, color=["condition"], wspace=1)

In [ ]:
adata_solo[adata_solo.obs["batch_id"].isin(["2"])].obs["sample_id"].value_counts()

In [ ]:
adata_solo.obs["sample_id"].value_counts()

In [ ]:
sc.pl.umap(
    adata_solo,
    color=["pct_counts_mt", "n_genes_by_counts"],
    cmap="inferno",
)

In [ ]:
sc.pl.umap(adata_solo, color=["total_counts", "leiden"], vmax=10000, legend_loc="on data")

In [ ]:
#Function to add a bolean mask ti label the barcode cell or EMTPY droplet
def annot_empty_droplets(adata, boolean):
    import numpy as np
    
    if "is_droplet" not in adata.obs.columns:
        adata.obs["is_droplet"] = "cell"

    is_droplet = np.where(
        (adata.obs["is_droplet"] == "cell") & boolean, "cell", "droplet"
    )
    return is_droplet

In [ ]:
# Remove empty droplets before scVI neighbors
adata_solo.obs["is_droplet"] = annot_empty_droplets(
    adata_solo,
    ~(
        (adata_solo.obs["leiden"].isin(["0","1","2","3","4","5","6"]))
        & (adata_solo.obs["total_counts"] < 2000)
        & (adata_solo.obs["n_genes_by_counts"] < 500)
    ),
)

In [ ]:
adata_solo.obs["is_droplet"].value_counts()

In [ ]:
## Zoom into a specific leiden cluster 
sc.pl.umap(
    adata_solo[adata_solo.obs["leiden"].isin(["0"])],
    color=["pct_counts_mt", "n_genes_by_counts"],
    cmap="inferno",
)

sc.pl.umap(adata_solo[adata_solo.obs["leiden"].isin(["0"])], color=["total_counts", "leiden"], vmax=20000)